# Windy grid world

In [4]:
#######################################################################
# Copyright (C)                                                       #
# 2016-2018 Shangtong Zhang(zhangshangtong.cpp@gmail.com)             #
# 2016 Kenta Shimada(hyperkentakun@gmail.com)                         #
# Permission given to modify the code as long as you keep this        #
# declaration at the top                                              #
#######################################################################
# This code was modified from the original code to better serve the purposes of our RL course at FGV.
############################################################################################
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import copy

# The problem
In this problem, we have a grid world  with a start state and a goal state. The agent can move in four directions: up, down, left, and right. The wind can blow the agent up in some columns. The agent receives a reward of -1 for each step, except when it reaches the goal state, where it receives a reward of 0. The agent can choose to move in any direction, but the wind will affect its movement. The goal is to find the optimal policy for the agent to reach the goal state in the fewest steps possible.

<img src="windy_grid.png">


In [5]:
# world height
WORLD_HEIGHT = 7

# world width
WORLD_WIDTH = 10

# wind strength for each column
WIND = [0, 0, 0, 1, 1, 1, 2, 2, 1, 0]

# possible actions
ACTION_UP = 0
ACTION_DOWN = 1
ACTION_LEFT = 2
ACTION_RIGHT = 3

# probability for exploration
EPSILON = 0.1

# Sarsa step size
ALPHA = 0.5

# reward for each step
REWARD = -1.0

START = [3, 0]
GOAL = [3, 7]
ACTIONS = [ACTION_UP, ACTION_DOWN, ACTION_LEFT, ACTION_RIGHT]

## Implementing the SARSA algorithm
We'll implement the SARSA algorithm to find the optimal policy for the windy grid world problem. The SARSA algorithm is an on-policy algorithm that uses the epsilon-greedy policy to explore the environment. The algorithm updates the Q-values based on the current state, action, reward, and next state and action. The goal is to find the optimal policy that maximizes the expected return in the long run.

In [6]:
def step(state, action):
    i, j = state
    if action == ACTION_UP:
        return [max(i - 1 - WIND[j], 0), j]
    elif action == ACTION_DOWN:
        return [max(min(i + 1 - WIND[j], WORLD_HEIGHT - 1), 0), j]
    elif action == ACTION_LEFT:
        return [max(i - WIND[j], 0), max(j - 1, 0)]
    elif action == ACTION_RIGHT:
        return [max(i - WIND[j], 0), min(j + 1, WORLD_WIDTH - 1)]
    else:
        assert False

# play for an episode
def episode(q_value):
    # track the total time steps in this episode
    time = 0

    # initialize state
    state = START

    # choose an action based on epsilon-greedy algorithm
    if np.random.binomial(1, EPSILON) == 1:
        action = np.random.choice(ACTIONS)
    else:
        values_ = q_value[state[0], state[1], :]
        action = np.random.choice([action_ for action_, value_ in enumerate(values_) if value_ == np.max(values_)])

    # keep going until get to the goal state
    while state != GOAL:
        next_state = step(state, action)
        if np.random.binomial(1, EPSILON) == 1:
            next_action = np.random.choice(ACTIONS)
        else:
            values_ = q_value[next_state[0], next_state[1], :]
            next_action = np.random.choice([action_ for action_, value_ in enumerate(values_) if value_ == np.max(values_)])

        # Sarsa update
        q_value[state[0], state[1], action] += \
            ALPHA * (REWARD + q_value[next_state[0], next_state[1], next_action] -
                     q_value[state[0], state[1], action])
        state = next_state
        action = next_action
        time += 1
    return time

In [7]:
# Figure 6.7, 1,000 runs may be enough, # of actions in state B will also affect the curves
def figure_6_3():
    q_value = np.zeros((WORLD_HEIGHT, WORLD_WIDTH, 4))
    episode_limit = 500

    steps = []
    ep = 0
    while ep < episode_limit:
        steps.append(episode(q_value))
        # time = episode(q_value)
        # episodes.extend([ep] * time)
        ep += 1

    steps = np.add.accumulate(steps)

    plt.plot(steps, np.arange(1, len(steps) + 1))
    plt.xlabel('Time steps')
    plt.ylabel('Episodes')

    plt.savefig('../images/figure_6_3.png')
    plt.close()

    # display the optimal policy
    optimal_policy = []
    for i in range(0, WORLD_HEIGHT):
        optimal_policy.append([])
        for j in range(0, WORLD_WIDTH):
            if [i, j] == GOAL:
                optimal_policy[-1].append('G')
                continue
            bestAction = np.argmax(q_value[i, j, :])
            if bestAction == ACTION_UP:
                optimal_policy[-1].append('↑')
            elif bestAction == ACTION_DOWN:
                optimal_policy[-1].append('↓')
            elif bestAction == ACTION_LEFT:
                optimal_policy[-1].append('←')
            elif bestAction == ACTION_RIGHT:
                optimal_policy[-1].append('→')
    print('Optimal policy is:')
    for row in optimal_policy:
        print(row)
    print('Wind strength for each column:\n{}'.format([str(w) for w in WIND]))
    
figure_6_3()

Optimal policy is:
['←', '→', '↓', '→', '→', '→', '→', '→', '→', '↓']
['→', '→', '→', '→', '→', '→', '→', '→', '→', '↓']
['→', '→', '↑', '→', '→', '→', '↑', '←', '→', '↓']
['→', '→', '→', '→', '→', '→', '→', 'G', '→', '↓']
['→', '→', '→', '→', '↑', '→', '↑', '↓', '←', '←']
['↓', '→', '→', '→', '→', '↑', '↑', '↓', '←', '↓']
['→', '→', '→', '→', '↑', '↑', '↑', '↑', '←', '←']
Wind strength for each column:
['0', '0', '0', '1', '1', '1', '2', '2', '1', '0']
